# E01

Train a trigram language model, i.e. take two characters as an input to predict the 3rd one. Feel free to use either counting or a neural net. Evaluate the loss; Did it improve over a bigram model?

In [1]:
words = open('../names.txt', 'r').read().splitlines()

In [2]:
len(words)

32033

In [3]:
words[:5]

['emma', 'olivia', 'ava', 'isabella', 'sophia']

The model will predict the next character given the two preceding characters.

In [4]:
# Each entry is a training example
for word in words[:2]:
    chs = '..'+word+'.'
    for ch1,ch2,ch3 in zip(chs,chs[1:],chs[2:]):
        print(ch1,ch2,ch3)

. . e
. e m
e m m
m m a
m a .
. . o
. o l
o l i
l i v
i v i
v i a
i a .


In [5]:
chars = sorted(list(set(''.join(words))))
len(chars)

26

Add the character that marks the beginning of string. This character will be used twice for when we want to indicate that we are at the beginning of a potential name.

In [6]:
ctoi = {c : i+1 for i,c in enumerate(chars)}
ctoi['.']=0

In [7]:
itoc = {i:c for c,i in ctoi.items()}

In [8]:
num_chars = len(ctoi.keys())
num_chars

27

Create a dictionary that takes two characters and returns an integer. This will be used when sampling from the model.

In [9]:
stoi = {}
for i0,c0 in sorted(itoc.items(), key=lambda kv: kv[0]):
    for i1,c1 in sorted(itoc.items(), key=lambda kv: kv[0]):
        #print((i0*num_chars) + i1,c0,c1)
        stoi[(c0,c1)] = (i0*num_chars) + i1

In [10]:
stoi[('.','.')], stoi[('z','z')]

(0, 728)

# Counting Model

In [11]:
import torch

Create a 729 x 27 array to hold the counts for each trigram. We initialize this with ones to "smooth" and ensure no issues if a zero is encountered when we take log.

In [12]:
N = torch.ones((num_chars*num_chars, num_chars), 
                dtype=torch.int64
               )
N.dtype

torch.int64

In [13]:
for word in words:
    chs = '..'+word+'.'
    for ch1,ch2,ch3 in zip(chs,chs[1:],chs[2:]):
        row = stoi[(ch1,ch2)]
        column = ctoi[ch3]
        N[row,column] += 1

In [14]:
N.shape

torch.Size([729, 27])

Normalize each row of N by dividing by the sum of counts in that row. So now each row of P should equal 1.

In [15]:
P = N.float()
P /=P.sum(dim=1,keepdim=True) 

In [16]:
P.sum(dim=1, keepdim=True).min(), P.sum(dim=1, keepdim=True).mean(), P.sum(dim=1, keepdim=True).max()

(tensor(1.0000), tensor(1.), tensor(1.0000))

Sample from the trigram model

In [17]:
N[1]

tensor([  1, 208, 191,  32, 367,  56,  22,  18,  92, 155,  28,  76, 633, 385,
        624,  11,  18,  10, 483, 195,  73, 153, 244,   7,  28, 174, 153])

In [18]:
P[0].shape

torch.Size([27])

## Sampling Method 1

In [19]:
g = torch.Generator().manual_seed(2147483647)
for _ in range(10):
    out=[]
    ix = 0 #start off with ..
    while True:
        p = P[ix]
        next_char_idx = torch.multinomial(p, 
                                          num_samples=1, 
                                          replacement=True, 
                                          generator=g
                                         ).item()
        next_char = itoc[next_char_idx]
        
        #tack on the next character
        out.append(next_char)
        
        if next_char_idx == 0:
            break

        # if only a single entry in out this means we are just starting out
        # so we add a dot to the first character sampled from the trigram
        # model
        ix = stoi[('.',out[-1])] if len(out)==1 else stoi[(out[-2],out[-1])]
    
    print(''.join(out))

ce.
bra.
jalius.
rochityharlonimittain.
luwak.
ka.
da.
samiyah.
javer.
gotai.


Another way to sample where we fill the out with the desired starting characters.

## Sampling Method 2

In [20]:
g = torch.Generator().manual_seed(2147483647)
for _ in range(20):
    out=['.','.']
    ix = stoi[(out[-2],out[-1])]
    while True:
        p = P[ix]
        next_char_idx = torch.multinomial(p, 
                                          num_samples=1, 
                                          replacement=True, 
                                          generator=g
                                         ).item()
        next_char = itoc[next_char_idx]
        
        #tack on the next character
        out.append(next_char)
        
        if next_char_idx == 0:
            break
        
        ix = stoi[(out[-2],out[-1])]
    
    print(''.join(out).replace('.',''))

ce
bra
jalius
rochityharlonimittain
luwak
ka
da
samiyah
javer
gotai
moriellavojkwuthda
kaley
maside
en
aviyah
fobspihiliven
tahlasuzusfxx
an
glhpynn
isan


## Sampling Uniformly at Random

The Trigram model does not seem like it is working i.e, the output does not seem very namelike. So let's replace sampling from P with sampling for the next character from a uniform distributions and see the results.

Now we see that the results are essentially garbage for the untrained model that is just picking the next character uniformly at random.

In [21]:
g = torch.Generator().manual_seed(2147483647)
for _ in range(50):
    out=[]
    ix = 0 #start off with ..
    while True:
        p = torch.ones(num_chars)/(1.0*num_chars)
        next_char_idx = torch.multinomial(p, 
                                          num_samples=1, 
                                          replacement=True, 
                                          generator=g
                                         ).item()
        next_char = itoc[next_char_idx]
        
        #tack on the next character
        out.append(next_char)
        
        if next_char_idx == 0:
            break

        # if only a single entry in out this means we are just starting out
        # so we add a dot to the first character sampled from the trigram
        # model
        ix = stoi[('.',out[-1])] if len(out)==1 else stoi[(out[-2],out[-1])]
    
    print(''.join(out))

cexzm.
zoglkurkicqzktyhwmvmzimjttainrlkfukzkktda.
sfcxvpubjtbhrmgotzx.
iczixqctvujkwptedogkkjemkmmsidguenkbvgynywftbspmhwcivgbvtahlvsu.
dsdxxblnwglhpyiw.
igwnjwrpfdwipkwzkm.
desu.
firmt.
gbiksjbquabsvoth.
kuysxqevhcmrbxmcwyhrrjenvxmvpfkmwmghfvjzxobomysox.
gbptjapxweegpfwhccfyzfvksiiqmvwbhmiwqmdgzqsamjhgamcxwmmk.
iswcxfmbalcslhy.
fpycvasvz.
bqzazeunschck.
wnkojuoxyvtvfiwksddugnkul.
fuwfcgjz.
abl.
j.
nuuutstofgqzubbo.
rdubpknhmd.
vhfacdvaaasjzjkdh.
gh.
frdhlhahflrklmlcugro.
pnxhayx.
vn.
gixgosfqn.
mempfnclfxtirbqhvjfdwhzymayerzqvmzjvtjuifbooocnkcxjkvsmjafciekxoraw.
.
veigtbcaamnef.
chfeukwowgsadjjkkswrcpawhoxskfikwbscynndmiuxxwoturzhqnsjdndsziocnbxiegzzulhnqdqwosi.
kdwnfjvmtthtpzbmdvvrvtptaqlhdnkj.
zxkcbczsrcagitwicvkcqiotgnvpllciqs.
uohjxnvxqikebadkdawdfwwha.
fqcnmrpoljlpjldyjehpprjppsmkzdhrmgyoadmsod.
dnvzcobtzfikidecxjhbmmjxqphvtedjbwkxzhisndnoauiycrdfetifkvzlzf.
ud.
ckndsgyldqbkcylrozgwkjgftrahdrnfapspdayna.
thavpgelvlfqxxsdabgxpyzv.
ikzvrykvyxhuj.
qkpwzuaics.
xxqubplmqguhbpnetz.
.
t

## Evaluate the loss

In [22]:
log_likelihood = 0.
n = 0
for word in words:
    chs = '..' + word + '.'
    for ch1,ch2,ch3 in zip(chs,chs[1:],chs[2:]):
        #print(ch1,ch2,ch3)
        prob = P[stoi[ch1,ch2],ctoi[ch3]]
        logprob = torch.log(prob)
        log_likelihood += logprob
        n += 1

print(f'{log_likelihood=:.4f}') 
print(f'{n=}')
nll = -log_likelihood
print(f'{nll/n=:.4f}')  

log_likelihood=-504653.0000
n=228146
nll/n=2.2120


The average negative log likelihood over the entire training set of the Bigram counting model was 2.4509 while that over the training set of the Trigram counting model is 2.21. Thus, we have an improvement.

# Neural Network

## Training Set

In [23]:
import torch.nn.functional as F

In [24]:
xs,ys = [],[]

for word in words:
    chs = '..' + word + '.'
    for ch1,ch2,ch3 in zip(chs,chs[1:],chs[2:]):
        ix1 = stoi[ch1,ch2]
        ix2 = ctoi[ch3]
        xs.append(ix1)
        ys.append(ix2)

# prefer to use torch.tensor instead of torch.Tensor
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print(f'number of examples: {num}')

# initialize the network  

g = torch.Generator().manual_seed(2147483647)
W = torch.randn((num_chars*num_chars,num_chars), generator=g, requires_grad=True) #single layer of 27 neurons each getting 27x27 inputs

number of examples: 228146


In [25]:
for k in range(400):
    xenc = F.one_hot(xs, num_classes = num_chars*num_chars).float()
    logits = xenc@W #log-counts
    counts = logits.exp() # exponentiate the logits to get fake counts
    probs = counts/counts.sum(1,keepdims=True)
    loss = (-(probs[torch.arange(num),ys]).log()).mean()

    if k%40==0:
        print(loss.item())
    
    #backward pass
    W.grad = None #More efficient than setting to zero directly. Lack of gradient is interpreted as zero by PyTorch
    loss.backward()
    
    #update
    W.data += -4*50 * W.grad    

print(loss.item())

3.792776346206665
2.44636607170105
2.3429062366485596
2.3829033374786377
2.2747690677642822
2.318384885787964
2.3451452255249023
2.244135856628418
2.263615131378174
2.2489423751831055
2.315772771835327


## Sample from the neural network

In [26]:
g = torch.Generator().manual_seed(2147483647)

for _ in range(5):
    out=[]
    ix = 0
    while True:
        xenc = F.one_hot(torch.tensor([ix]), num_classes = num_chars*num_chars).float()
        logits = xenc@W #log-counts
        counts = logits.exp() # exponentiate the logits to get fake counts
        p = counts/counts.sum(1,keepdims=True)
        #print(p)
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itoc[ix])
        if ix == 0:
            break
        ix = stoi[('.',out[-1])] if len(out)==1 else stoi[(out[-2],out[-1])]
    print(''.join(out))

ce.
bra.
emon.
raila.
kaydemmilistona.


## Evaluate the loss

In [27]:
log_likelihood = 0.
n = 0
for word in words:
    chs = '..' + word + '.'
    for ch1,ch2,ch3 in zip(chs,chs[1:],chs[2:]):
        ix = stoi[ch1,ch2]
        xenc = F.one_hot(torch.tensor([ix]), num_classes = num_chars*num_chars).float()
        logits = xenc@W #log-counts
        counts = logits.exp() # exponentiate the logits to get fake counts
        p = counts/counts.sum(1,keepdims=True)
        #print(p.shape)
        #print(ch1,ch2,ch3)
        prob = p[0,ctoi[ch3]]
        logprob = torch.log(prob)
        log_likelihood += logprob
        n += 1

print(f'{log_likelihood=:.4f}') 
print(f'{n=}')
nll = -log_likelihood
print(f'{nll/n=:.4f}')  

log_likelihood=-511720.9688
n=228146
nll/n=2.2430
